# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azure.ai.ml import MLClient
import azure.ai.ml as aml

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
print("azure-ai-ml versoins:", aml.__version__)

SDK version: 1.61.0
azure-ai-ml versoins: 1.31.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure'
project_folder = './'

quick-starts-ws-295371
aml-quickstarts-295371
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
from azure.identity import DefaultAzureCredential

experiment=Experiment(ws, experiment_name)
experiment

subscription_id="3d1a56d2-7c81-4118-9790-f85d1acf0c77"
resource_group="aml-quickstarts-295371"
workspace_name="quick-starts-ws-295371"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print("Connected to: ", ml_client.workspace_name)

Connected to:  quick-starts-ws-295371


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
data_asset = ml_client.data.get(name="heart-failure-dataset", version="1")

print("name", data_asset.name)
print("version", data_asset.version)
print("type", data_asset.type)
print("path", data_asset.path)

name heart-failure-dataset
version 1
type mltable
path azureml://subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295371/workspaces/quick-starts-ws-295371/datastores/workspaceblobstore/paths/UI/2026-02-03_091333_UTC/heart_failure_clinical_records_dataset.csv/


In [5]:
# Choose a name for your CPU cluster
cluster_name = "cluster00"

compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    
if compute_target:
    print('Found existing cluster, use it.')
else: 
    print("Not found under name " + cluster_name)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azure.ai.ml.automl import classification
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

job = classification(
    compute="cluster00",
    training_data=Input(type=AssetTypes.MLTABLE, path="azureml:heart-failure-dataset:1"),
    target_column_name="DEATH_EVENT",
    primary_metric="accuracy"
)

job.set_limits(timeout_minutes=30)
job.set_limits(max_trials=20)
job.set_limits(max_concurrent_trials=5)

created = ml_client.jobs.create_or_update(job)
print("Job name: ", created.name)

ml_client.jobs.stream(created.name)
parent_job_id = created.name


Job name:  cyan_airport_f0jz6x4fl9
RunId: cyan_airport_f0jz6x4fl9
Web View: https://ml.azure.com/runs/cyan_airport_f0jz6x4fl9?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295371/workspaces/quick-starts-ws-295371

Execution Summary
RunId: cyan_airport_f0jz6x4fl9
Web View: https://ml.azure.com/runs/cyan_airport_f0jz6x4fl9?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-295371/workspaces/quick-starts-ws-295371



/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.8.1) and child packages mlflow-skinny (3.5.0) are different. This may lead to unexpected behavior. Please install the same version of all MLflow packages.
  mlflow.mismatch._check_version_mismatch()


In [7]:
parent_job = ml_client.jobs.get(parent_job_id)
print(parent_job.status)

ml_client.jobs.download(
    name=parent_job_id,
    output_name="best_model",
    download_path="./best_model"
)
print("Downloaded")

Completed
Downloaded


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azure.ai.ml.entities import Model

mlflow_dir = None

for root, dirs, files in os.walk("./best_model"):
    if "MLmodel" in files:
        mlflow_dir = root
        break

print("model in dir: ", mlflow_dir)

model = Model(
    name="hf-best-model",
    path=mlflow_dir,
    type="mlflow_model"
)

registered = ml_client.models.create_or_update(model)
print("Registered: ", registered.name,  " version: ", registered.version)

model in dir:  ./best_model/named-outputs/best_model
Registered:  hf-best-model  version:  3


Uploading best_model (0.73 MBs): 100%|██████████| 730878/730878 [00:00<00:00, 23872942.29it/s]




In [9]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

endpoint = ManagedOnlineEndpoint(
    name="hf-endpoint",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("Endpoint created")

Endpoint created


In [12]:
from azure.ai.ml.entities import ManagedOnlineDeployment

deployment = ManagedOnlineDeployment(
    name="heart-failure-consume00",
    endpoint_name="hf-endpoint",
    model="hf-best-model:3",
    instance_type="Standard_DS3_v2",
    instance_count=1
)

ml_client.online_deployments.begin_create_or_update(deployment)
print("Deployment created")

Check: endpoint hf-endpoint exists


.Deployment created
..............................................................................................................................................

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

NameError: name 'pipeline_run' is not defined

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Save the best model
best_automl_run, best_automl_model= remote_run.get_output()
best_automl_model = best_automl_run.register_model(model_name="heart-failure-best-model", model_path="outputs/")

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
environment = best_automl_run.get_environment()
entry_script='scoring.py'

In [ ]:
inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "bestmodeldeployv4", [best_automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
%run endpoint.py

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
